In [1]:
from bs4 import *
import re
import urllib.request as urllib2
from urllib.request import urlopen
from urllib.parse  import urljoin
import requests
import unicodedata
import math

In [2]:
url = 'https://mafiaworldtour.com/user/14110/show'

In [3]:
result = [[0, 0, 0.0], [0, 0, 0.0], [0, 0, 0.0], [0, 0, 0.0], 0]

In [4]:
#блок с ссылками
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
name = soup.select('#user-general-info > div:nth-child(1) > div.col-md-8 > div.table-responsive > div > div:nth-child(2) > div.col-md-9 > span')
#ник
player_name = name[0].text
player_name

'TurboKazah'

In [5]:
#получаем все ссылки
links = soup('a', class_='btn btn-sm btn-danger')
tournament_links = []
for i in range(len(links)):
    tournament_links.append(links[i]['href']) 
len(tournament_links)

2

In [6]:
def all_games(url):
    #делим страницу на игры
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    game_panels = soup.find_all('div', class_='panel panel-default')
    
    for k in range(len(game_panels)):
        panel = game_panels[k]
        
        #кто победил
        panel_title = panel.select_one('div > div > h4 > a')
        if panel_title['class'][1] == 'mafia-wins':
            city_wins = False
        else:
            city_wins = True
        
        #находим нашего игрока
        players = panel('div', class_ = 'game-result-block')
        for i in range(len(players)):
            a = players[i](class_='highlighed-result')
            if a!=[]:
                player_number = i
        player_box = players[player_number]
        #print(player_box.prettify())
        
        #получаем его результат
        if player_box(class_ = 'is-killed-first'):
            result[4] += 1

        points = player_box.find(class_ = 'score-cell')
        bonus_points = float(re.split('/', points.text)[-1])
        role = player_box('i')[-1]

        if role['class'] == ['icon-red']:
            result[0][0] += 1
            if city_wins == True:
                result[0][1] += 1
            result[0][2] += bonus_points

        elif role['class'] == ['icon-sheriff']:
            result[1][0] += 1
            if city_wins == True:
                result[1][1] += 1
            result[1][2] += bonus_points

        elif role['class'] == ['icon-black']:
            result[2][0] += 1
            if city_wins == False:
                result[2][1] += 1
            result[2][2] += bonus_points

        elif role['class'] == ['icon-don']:
            result[3][0] += 1
            if city_wins == False:
                result[3][1] += 1
            result[3][2] += bonus_points
    return(len(game_panels))

#test

In [7]:
gamecount = 0
for i in range(len(tournament_links)):
    gamecount += all_games(tournament_links[i])
gamecount

26

In [8]:
print(player_name)
print('Красные:', result[0][0], '/', result[0][1], '/', "%.2f" %(result[0][2]))
print('Шериф:', result[1][0], '/', result[1][1], '/', "%.2f" %(result[1][2]))
print('Чёрные:', result[2][0], '/', result[2][1], '/', "%.2f" %(result[2][2]))
print('Дон:', result[3][0], '/', result[3][1], '/', "%.2f" %(result[3][2]))
print('ПУ:', result[4])

TurboKazah
Красные: 16 / 4 / 1.10
Шериф: 2 / 0 / 0.00
Чёрные: 6 / 3 / 0.20
Дон: 2 / 1 / 0.20
ПУ: 0


In [9]:
with open(f'{player_name}.txt', 'w') as f:
    print(player_name, file=f)
    print('Красные:', result[0][0], '/', result[0][1], '/', "%.2f" %(result[0][2]), file=f)
    print('Шериф:', result[1][0], '/', result[1][1], '/', "%.2f" %(result[1][2]), file=f)
    print('Чёрные:', result[2][0], '/', result[2][1], '/', "%.2f" %(result[2][2]), file=f)
    print('Дон:', result[3][0], '/', result[3][1], '/', "%.2f" %(result[3][2]), file=f)
    print('ПУ:', result[4], file = f)
result = [[0, 0, 0.0], [0, 0, 0.0], [0, 0, 0.0], [0, 0, 0.0], 0]